# Income Analysis

This project aims to use a dataset of incomes of adults from the 1994 Census to create models that can be used to predict whether an individual makes an annual income of greater than $50,000 or not based on characteristics about the individual. Various supervised learning models will be explored to determine how these models perform on this classification task.

The goal of the project is to determine which sociodemographic characteristics about an individual have an influence on their annual income and by how much these characteristics influence the annual income. Doing so could provide insights into social inequalities regarding an individual's income.

## Dataset

The dataset used for this project is titled "Adult" and is from the University of California Irvine's Machine Learning Repository.

Becker, B., & Kohavi, R. (1996). Adult. UCI Machine Learning Repository. https://doi.org/10.24432/C5XW20